In [66]:
import pandas as pd
import os
import numpy as np 
import altair as alt
import pycountry

## Chart Theme Config

In [2]:
def e4e_basic():
    return {
        'config': {
            'axisX': {
                'labelPadding': 6,
                'tickOpacity': 0,
                'title': None,
                'grid': False,
                'ticks': False,
                'labelSeparation': 10
            },
            'axisY': {
                'title': None,
                'grid': False,
                'tickOpacity': 0,
            },
            'title': {
                'fontSize': 16,
                'subtitleFontSize': 12,
                'subtitleFontStyle': 'italic',
                'anchor': 'start',
                'color': 'black'
            },
            'view': {
                'stroke': 'transparent'
            }
        }
    }

# Register the custom theme under a chosen name
alt.themes.register('e4e_basic', e4e_basic)

# Enable the theme
alt.themes.enable('e4e_basic')

ThemeRegistry.enable('e4e_basic')

## Chart 1: Gov Spending as a % of GDP (long-term)

In [64]:
df = pd.read_csv("data/imf_gov_spending_gdp.csv")
df = df.iloc[1:]
df.columns = ["Country"] + list(df.columns[1:])

# replace China, People's Republic of with China
df["Country"] = df["Country"].str.replace("China, People's Republic of", "China")

df = df.query("Country == 'China'")

df = df.melt(id_vars=["Country"], var_name="Year", value_name="Value")

df["Value"] = pd.to_numeric(df["Value"], errors="coerce")
df = df.dropna()
df["Value"] = df["Value"] / 100

df["Year"] = pd.to_datetime(df["Year"], format="%Y")

spend_df = df.copy()

base = alt.Chart(df).encode(
    x=alt.X("Year:T", axis=alt.Axis(labelAngle=0)),
    y=alt.Y("Value:Q", title="", axis=alt.Axis(format="%")),
)

line = base.mark_line().encode(
    color=alt.value("rgb(9, 32, 95)")
)


label_1 = base.transform_filter("year(datum.Year) == 2022").mark_text(
    align="left",
    baseline="middle",
    dx=5,
    dy=5,
    fontSize=12,
).encode(
    text=alt.Text("Value:Q", format=".0%"),
    color=alt.value("rgb(9, 32, 95)")
)

label_2 = base.transform_filter("year(datum.Year) == 2022").mark_text(
    align="left",
    baseline="middle",
    dx=5,
    dy=-5,
    fontSize=12,
    text=alt.expr("'Spending'")
).encode(

    color=alt.value("rgb(9, 32, 95)")
)



chart = line + label_1 + label_2

chart = chart.properties(
    title = alt.TitleParams(
            "China: Government Spending",
    subtitle=["Central Government, % of GDP", "Source: IMF", ""],
    ),

    width=400,
    height=300
)

chart.display()

chart.save("charts/china/gov_spending_gdp.json")
chart.save("charts/china/gov_spending_gdp.png", scale_factor=2.0)
chart.save("charts/china/gov_spending_gdp.svg")

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


## Chart: Spending vs Revenue

In [61]:
df = pd.read_csv("data/imf_gov_revenue_gdp.csv")
df = df.iloc[1:]
df.columns = ["Country"] + list(df.columns[1:])

df["Country"] = df["Country"].str.replace("China, People's Republic of", "China")

df = df.query("Country == 'China'")


df = df.melt(id_vars=["Country"], var_name="Year", value_name="Value")

df["Value"] = pd.to_numeric(df["Value"], errors="coerce")
df = df.dropna()
df["Value"] = df["Value"] / 100

df["Year"] = pd.to_datetime(df["Year"], format="%Y")

rev_df = df.copy()

df = pd.merge(spend_df, rev_df, on=["Country", "Year"], suffixes=("_spend", "_rev"))

df = df.rename(columns={"Value_spend": "Spending", "Value_rev": "Revenue"})
df = df.melt(id_vars=["Country", "Year"], var_name="Series", value_name="Value")
df = df.query("Year >= '1948-01-01'")

base = alt.Chart(df).encode(
    x=alt.X("Year:T", axis=alt.Axis(labelAngle=0)),
    y=alt.Y("Value:Q", title="", axis=alt.Axis(format="%")),
)

line = base.mark_line().encode(
    color=alt.Color("Series:N", scale=alt.Scale(range=["rgb(9, 32, 95)", "rgb(119, 22, 8)"]), legend=None),
)

labels_1 = base.transform_filter("year(datum.Year) == 2022").mark_text(
    align="left",
    baseline="middle",
    dx=5,
    dy=-5,
    fontSize=12,
).encode(
    text="Series:N",
    color=alt.Color("Series:N", scale=alt.Scale(range=["rgb(9, 32, 95)", "rgb(119, 22, 8)"])),
)

labels_2 = base.transform_filter("year(datum.Year) == 2022").mark_text(
    align="left",
    baseline="middle",
    dx=5,
    dy=5,
    fontSize=12,
).encode(
    text=alt.Text("Value:Q", format=".0%"),
    color=alt.Color("Series:N", scale=alt.Scale(range=["rgb(9, 32, 95)", "rgb(119, 22, 8)"])),
)




chart = line + labels_1 + labels_2



chart = chart.properties(
    title = alt.TitleParams(
            "China: Surplus or Shortfall?",
    subtitle=["Central Government, % of GDP", "Source: IMF", ""],
    ),

    width=400,
    height=300
)

chart.save("charts/china/surplus_shortfall.json")
chart.save("charts/china/surplus_shortfall.png", scale_factor=2.0)
chart.save("charts/china/surplus_shortfall.svg")

chart.display()


/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

alt.LayerChart(...)

## Chart: Spending by Function

Data from https://chinapower.csis.org/making-sense-of-chinas-government-budget/

In [63]:
df = pd.read_csv("data/budget_allocation.csv")

df["Value"] = df.Share.str.replace("%", "").astype(float) / 100

base = alt.Chart(df).encode(
    x=alt.X("Value:Q", title="", axis=alt.Axis(format="%")),
    y=alt.Y("Category:N", title="", sort="-x"),
)

bar = base.mark_bar(color="rgb(9, 32, 95)")

labels = bar.mark_text(
    align="left",
    baseline="middle",
    dx=3,
    fontSize=12,
).encode(
    text=alt.Text("Value:Q", format=".0%"),
)

chart = bar + labels

chart = chart.properties(
    title = alt.TitleParams(
            "China: Government Expenditure",
    subtitle=["2022, Central Government, % of Budget", "Source: Chinese Ministry of Finance via CSIS", ""],
    ),

    width=300,
    height=400
)


chart.save("charts/china/gov_expenditure_category.json")
chart.save("charts/china/gov_expenditure_category.png", scale_factor=2.0)
chart.save("charts/china/gov_expenditure_category.svg")


chart.display()



/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
WARN Domains that should be unioned has conflicting sort properties. Sort will be set to true.
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = 

alt.LayerChart(...)

## Chart: Debt (% of GDP)

In [65]:
df = pd.read_csv("data/imf_central_gov_debt_gdp.csv")
df = df.iloc[1:]
df.columns = ["Country"] + list(df.columns[1:])

df["Country"] = df["Country"].str.replace("China, People's Republic of", "China")
df = df.query("Country == 'China'")


df = df.melt(id_vars=["Country"], var_name="Year", value_name="Value")

df["Value"] = pd.to_numeric(df["Value"], errors="coerce")/100
df = df.dropna()


base = alt.Chart(df).encode(
    x=alt.X("Year:T", axis=alt.Axis(labelAngle=0)),
    y=alt.Y("Value:Q", title="", axis=alt.Axis(format="%")),
)

line = base.mark_line().encode(
    color=alt.value("rgb(9, 32, 95)")
)


labels = base.transform_filter("year(datum.Year) == 2028").mark_text(
    align="left",
    baseline="middle",
    dx=5,
    dy=-5,
    fontSize=12,
    text=alt.expr("'Debt'")
).encode(

    color=alt.value("rgb(9, 32, 95)")
)

labels_2 = base.transform_filter("year(datum.Year) == 2028").mark_text(
    align="left",
    baseline="middle",
    dx=5,
    dy=5,
    fontSize=12,
).encode(
    text=alt.Text("Value:Q", format=".0%"),
    color=alt.value("rgb(9, 32, 95)")
)

# shade >2024 to indicate forecast
shade = alt.Chart(pd.DataFrame({"Year": "2024-01-01", "Year_end": "2028-01-01"}, index=[0])).mark_rect(
    color="rgb(9, 32, 95)",
    opacity=0.1
).encode(
    x="Year:T",
    x2="Year_end:T",
    y=alt.value(0),
    y2=alt.value(300)
)


chart = line + labels + shade + labels_2

chart = chart.properties(
    title = alt.TitleParams(
            "China: Government Debt",
    subtitle=["Central Government, % of GDP", "Source: IMF, Forecast Shaded", ""],
    ),

    width=400,
    height=300
)

chart.display()

chart.save("charts/china/gov_debt_gdp.json")
chart.save("charts/china/gov_debt_gdp.png", scale_factor=2.0)
chart.save("charts/china/gov_debt_gdp.svg")


/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.LayerChart(...)

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/finn/anaconda3/lib/python3.11/

In [48]:
shade

/Users/finn/anaconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)